# <b> Chapter 9

### For some of the next chapters, I'll be using the public Data on the Revenue of the Brazilian State of Goiás

In [32]:
from datetime import datetime
import pandas as pd
import pytz
import os
import re

## Getting the Data

In [33]:
# Loading the data

root = os.getcwd()
paths = [
    root + "\\" + file
    for file in os.listdir()
    if re.match("ReceitasDetalhadas.*.csv", file)
]
if paths:
    df = pd.DataFrame()
    for i, path in enumerate(paths):
        with open(path, "r", encoding="utf-8") as f:
            data = pd.read_csv(f, index_col=False)
            df = pd.concat([df, data])
    df.columns

    # Formatting the dates

    t = ["int32" if re.match("(COD.*|RECEITA.*)", c) else "string" for c in df.columns]
    df = df.astype({k: v for k, v in zip([*df.columns], t)})
    df["MES"] = df["MES"].map(
        {
            "Janeiro": "01",
            "Fevereiro": "02",
            "Março": "03",
            "Abril": "04",
            "Maio": "05",
            "Junho": "06",
            "Julho": "07",
            "Agosto": "08",
            "Setembro": "09",
            "Outubro": "10",
            "Novembro": "11",
            "Dezembro": "12",
        }
    )
    date_parser = (
        lambda x: pytz.timezone("America/Sao_Paulo")
        .localize(datetime.strptime(x, "%Y/%m"))
        .isoformat()
    )
    df["DATA"] = df["ANO"] + "/" + df["MES"]
    df["DATA"] = pd.to_datetime(df["DATA"].map(date_parser))
    df = df.drop(columns=["ANO", "MES", "ANO_MES"], axis=1)
    df.to_csv(f"{root}\\ReceitasGoias2022", sep=";", index=False)